In [145]:
import pandas as pd
import numpy as np
import os
from os import listdir
from os.path import isfile, join

In [146]:
year = '2019'

def gen_data_path(year, data_path):
    return f'{data_path}/{year}'

data_path_2019 = gen_data_path(year, data_path)

In [147]:
def concat_dfs(data_path_year):
    filenames = [f for f in listdir(data_path_year) if isfile(join(data_path_year, f))]
    frames = []
    for fn in filenames:
        data = pd.read_csv(f'{data_path_year}/{fn}', error_bad_lines=False)
        frames.append(data)
    data_all = pd.concat(frames, ignore_index=True).reset_index(drop=True)
    
    return data_all

data_2019 = concat_dfs(data_path_2019)

/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3242: DtypeWarning: Columns (0,5,6,7) have mixed types. Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):
/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3242: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):


In [148]:
def clean_zipcode(df):
    drop_indices = []
    for index, zipcode in enumerate(df['zip code']):
        zipcode = str(zipcode)

        if not zipcode.isdigit():
            drop_indices.append(index)
        
        if len(zipcode)<5:
            drop_indices.append(index)
    
    df_cleaned = df.drop(drop_indices)\
                   .reset_index(drop=True)
    
    df_cleaned['zip code'] = df_cleaned['zip code'].astype(str)\
                                                   .apply(lambda x: x[:5])

    return df_cleaned

data_2019_cleaned = clean_zipcode(data_2019)

In [149]:
def gen_summary(df_cleaned, year):
    data_grouped = df_cleaned.groupby(['zip code']).agg({
        'amount': ['sum', 'mean', 'median', 'count']
    })

    data_grouped.columns = [f'total donations {year}', 
                            f'average donations {year}', 
                            f'median donations {year}', 
                            f'number of donations {year}']

    data_grouped = data_grouped.reset_index()
    
    data_final = pd.merge(data_grouped, 
                          df_cleaned[['zip code', 'state']].drop_duplicates(subset='zip code'), 
                          on='zip code', 
                          how='left')
    
    return data_final

In [153]:
def summary_pipeline(year, data_path):
    
    data_path_year = gen_data_path(year, data_path)
    data_year = concat_dfs(data_path_year)
    data_year_cleaned = clean_zipcode(data_year)
    data_summary = gen_summary(data_year_cleaned, year)
    
    return data_summary

In [157]:
summary_pipeline('2017-2018', data_path).columns[:-1]

Index(['zip code', 'total donations 2017-2018', 'average donations 2017-2018',
       'median donations 2017-2018', 'number of donations 2017-2018'],
      dtype='object')

In [154]:
years = [y for y in listdir(data_path) if '.DS_Store' not in y]
years

['2011-2012', '2015-2016', '2009-2010', '2019', '2013-2014', '2017-2018']

In [160]:
pluto = pd.read_csv('../data/project/All_Properties PLUTO_New_Data.csv', index_col=0)
pluto.ZIP = pluto.ZIP.astype(str)

In [164]:
for year in years:
    df_sum = summary_pipeline(year, data_path)
    df_sum.to_csv(f'../data/donations_summary/summary_{year}.csv', index=False)
    df_sum.rename(columns={'zip code': 'ZIP'}, inplace=True)
    
    pluto = pd.merge(pluto, 
                     df_sum[df_sum.columns[:-1]].drop_duplicates(subset='ZIP'),
                     on='ZIP',
                     how='left')

In [165]:
pluto

,ADDRESS,BOROUGH,BLOCK,LOT,LONGITUDE,LATITUDE,ZIP,REIS SUBMARKET,SCHOOL DIS,OWNER TYPE,...,median donations 2019_y,number of donations 2019_y,total donations 2013-2014_y,average donations 2013-2014_y,median donations 2013-2014_y,number of donations 2013-2014_y,total donations 2017-2018_y,average donations 2017-2018_y,median donations 2017-2018_y,number of donations 2017-2018_y
0,8-03 117 STREET,Queens,3935,3,-73.850386,40.789905,11356,Queens County,25.0,NaN,...,50.0,473.0,37150.0,1375.925926,500.0,27.0,70888.0,98.182825,30.0,722.0
1,216 FRANKLIN AVENUE,Staten Island,63,83,-74.091540,40.640063,10301,Staten Island,31.0,NaN,...,27.0,730.0,66514.0,496.373134,300.0,134.0,289263.0,141.865130,35.0,2039.0
2,5102 POST ROAD,Bronx,5825,1502,-73.897723,40.895985,10471,Bronx County,10.0,NaN,...,25.0,1414.0,221681.0,1274.028736,500.0,174.0,867516.0,206.158745,27.0,4208.0
3,2168 WEST 8 STREET,Brooklyn,7118,37,-73.979443,40.594658,11223,Kings County,21.0,NaN,...,30.0,699.0,76854.0,835.369565,500.0,92.0,167546.0,156.438842,27.0,1071.0
4,107-46 90 STREET,Queens,9152,28,-73.847597,40.676988,11417,Queens County,27.0,NaN,...,27.0,230.0,941.0,235.250000,235.5,4.0,17725.0,68.701550,41.0,258.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
780116,1639 LURTING AVENUE,Bronx,4100,19,-73.852402,40.846891,10461,Bronx County,11.0,NaN,...,37.0,362.0,28425.0,861.363636,500.0,33.0,55215.0,96.026087,35.0,575.0
780117,2147 WEST 5 STREET,Brooklyn,7122,65,-73.976001,40.595456,11223,Kings County,21.0,NaN,...,30.0,699.0,76854.0,835.369565,500.0,92.0,167546.0,156.438842,27.0,1071.0
780118,51-19 69 PLACE,Queens,2458,24,-73.894842,40.734864,11377,Queens County,24.0,NaN,...,27.0,1394.0,23910.0,508.723404,300.0,47.0,189015.0,94.839438,30.0,1993.0
780119,133 WINTER AVENUE,Staten Island,43,62,-74.086385,40.638805,10301,Staten Island,31.0,NaN,...,27.0,730.0,66514.0,496.373134,300.0,134.0,289263.0,141.865130,35.0,2039.0


In [166]:
data_path

'../data/Political_contributions_CSV'